# DataSens E1_v1 — 03_ingest_sources

- Objectifs: insérer données simulées (seed) pour tests
- Prérequis: 02_schema_create
- Sortie: lignes de démonstration dans `document`
- Guide: docs/GUIDE_TECHNIQUE_E1.md



> Notes:
> - Cette étape peuple la base avec un échantillon minimal pour tester CRUD.
> - `executescript` exécute plusieurs INSERT d’un coup (référentiels + un document).
> - Le `document` est rattaché à un `flux` pour la traçabilité.
> - L’opération (`DB_SEED`) est inscrite dans le fichier de versioning.


In [ ]:
# DataSens E1_v1 - 03_ingest_sources
# 🌱 Bootstrap: données de test (repris du monolithe)
import sqlite3
from datetime import datetime
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[2] if (Path.cwd().name.startswith('datasens_')) else Path.cwd()
DB_PATH = PROJECT_ROOT / "datasens" / "datasens.db"
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.executescript("""
INSERT INTO type_donnee (libelle, description, frequence_maj, categorie_metier)
VALUES ('Fichier','CSV Kaggle','hebdomadaire','descriptive');

INSERT INTO source (id_type_donnee, nom, url, fiabilite)
VALUES (1,'Kaggle CSV – FR textes','https://kaggle.com/xxx',0.8);

INSERT INTO flux (id_source, date_collecte, format, manifest_uri)
VALUES (1,'2025-10-27T10:00:00','csv','data/raw/kaggle/sample.csv');

INSERT INTO document (id_flux, titre, texte, langue, date_publication)
VALUES (1,'JO 2024 : fierté nationale','Les JO ont boosté le moral','fr','2024-07-27');
""")

conn.commit()
with open(PROJECT_ROOT / "README_VERSIONNING.md", "a", encoding="utf-8") as f:
    f.write(f"- **{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}** | `DB_SEED` | Insertion du jeu de données minimal\n")
print("🌱 Données de test insérées.")
